In [ ]:
# For colab
!pip uninstall tensorflow -y
!pip install tensorflow==2.3.1 tensorflow-quantum
!rm -rf quantum-gans
!git clone https://github.com/WiktorJ/quantum-gans
!cd quantum-gans; pip install .

In [1]:
from typing import Any

import sympy
import math
import tensorflow as tf
import tensorflow_quantum as tfq

import cirq
import numpy as np
from cirq import GridQubit, ops
from qsgenerator.qugans import circuits
from qsgenerator.qugans.training import Trainer, CustomSchedule
from qsgenerator.phase.circuits import build_ground_state_circuit
from qsgenerator.phase.analitical import construct_hamiltonian, get_theta_v, get_theta_w, get_theta_r

In [2]:
generator_layers = 2
discriminator_layers = 2
data_bus_size = 3

In [3]:
gen, gs, disc, ds, ls, data_qubits, out_qubit = circuits.build_gan_circuits(generator_layers, discriminator_layers, data_bus_size)

In [4]:
real, real_symbols = build_ground_state_circuit(qubits=data_qubits)

In [5]:
pure_gen = gen.copy()
gen.append([disc])

In [6]:
pure_real = real.copy()
real.append([disc])

In [7]:
print("REAL GROUND STATE: ", pure_real)

REAL GROUND STATE:  (0, 0): ───H───@─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
               │
(0, 1): ───────┼─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────U3((theta_w, 0, 0))───X───U3((theta_w, 0, 0))───U3((theta_v, 0, 0))───X───U3((theta_v, 0, 0))───
               │                                                                                                                                                                                                                                                               

In [8]:
print("GENERATOR:", pure_gen)

GENERATOR: (0, 2): ───Rx(g0)───Rz(g4)───ZZ────────────────────────Rx(g11)───Rz(g15)───ZZ──────────────────────────
                             │                                             │
(0, 3): ───Rx(g1)───Rz(g5)───ZZ^g8────────────ZZ───────Rx(g12)───Rz(g16)───ZZ^g19─────────────ZZ───────
                                              │                                               │
(0, 4): ───Rx(g2)───Rz(g6)────────────ZZ──────ZZ^g10───Rx(g13)───Rz(g17)─────────────ZZ───────ZZ^g21───
                                      │                                              │
(0, 5): ───Rz(l)────Rx(g3)───Rz(g7)───ZZ^g9────────────Rz(l)─────Rx(g14)───Rz(g18)───ZZ^g20────────────


In [9]:
print("DISCRIMINATOR:", disc)

DISCRIMINATOR: (0, 0): ───Rx(d0)───Rz(d5)────────────ZZ────────────────Rx(d14)───Rz(d19)─────────────ZZ────────────────
                                      │                                               │
(0, 1): ───Rz(l)────Rx(d1)───Rz(d6)───ZZ^d10───ZZ───────Rz(l)─────Rx(d15)───Rz(d20)───ZZ^d24───ZZ───────
                                               │                                               │
(0, 2): ───Rx(d2)───Rz(d7)───ZZ────────────────ZZ^d12───Rx(d16)───Rz(d21)───ZZ─────────────────ZZ^d26───
                             │                                              │
(0, 3): ───Rx(d3)───Rz(d8)───ZZ^d11───ZZ────────────────Rx(d17)───Rz(d22)───ZZ^d25────ZZ────────────────
                                      │                                               │
(0, 4): ───Rx(d4)───Rz(d9)────────────ZZ^d13────────────Rx(d18)───Rz(d23)─────────────ZZ^d27────────────


In [10]:
np.random.seed(0)
eps = 1e-2
init_gen_weights = np.array([np.pi] + [0] * 21) + \
                   np.random.normal(scale=eps, size=(22,))
init_disc_weights = np.random.normal(size=(28,))

gen_weights = tf.Variable(init_gen_weights, dtype=tf.float32)
disc_weights = tf.Variable(init_disc_weights, dtype=tf.float32)

In [11]:
learning_rate = CustomSchedule()

opt = tf.keras.optimizers.Adam(learning_rate, beta_1=0.9, beta_2=0.98, 
                                     epsilon=1e-9)

In [12]:
g_provider = lambda: 0

In [13]:
trainer = Trainer(g_provider, data_bus_size, disc, gen, real, out_qubit, ds, gs, real_symbols, ls)

In [14]:
epochs = 50
disc_iteration = 100
gen_iteration = 2
trainer.train(disc_weights,
      gen_weights, 
      opt, 
      epochs=epochs, 
      disc_iteration=disc_iteration, 
      gen_iteration=gen_iteration)

ValueError: Cannot serialize op <qsgenerator.phase.circuits.U3 object at 0x144f09370>.on(cirq.GridQubit(0, 2)) of type <class 'qsgenerator.phase.circuits.U3'>

In [ ]:
g = 0
ground_state = get_ground_state_for_g(g)
rad = map_to_radians(g)

trained_disc_weights = tf.Variable(np.array([-245.17047   ,  -95.07969   , -342.84647   , -422.7227    ,
       -183.86722   , -279.45145   , -208.48354   , -126.94892   ,
         99.149605  , -276.1524    ,  -82.480415  , -197.35036   ,
       -115.24699   , -111.50772   , -238.86653   , -150.12883   ,
       -271.65726   , -414.5756    , -183.14653   ,   97.29306   ,
       -120.5858    ,   -4.429718  ,  127.67174   ,  118.280525  ,
        305.74625   ,   -0.44963428,   17.841654  ,   93.42311   ]), dtype=tf.float32)

trained_gen_weights = tf.Variable(np.array([-44.918114 , -33.19301  , -65.66357  ,   7.7173038, -21.041788 ,
       -65.616486 , -47.297474 ,  62.851395 , -18.168066 , -21.922869 ,
        52.146767 , -70.66162  , -11.20047  ,  13.477829 , -24.01582  ,
        82.88633  , -82.02305  , -73.62559  ,  11.767767 ,  -3.0552819,
       -37.224037 ,  -1.1020341, rad]), dtype=tf.float32)


state_vector = tfq.layers.State()(pure_gen, symbol_names=gs + (ls,), symbol_values=tf.reshape(trained_gen_weights, (1, trained_gen_weights.shape[0]))).values.numpy()#%%

traced_out_state = cirq.wavefunction_partial_trace_as_mixture(state_vector, [0,1,2])

In [ ]:
prob, highest_prob_state = max(traced_out_state)
print(f"Max prob state has prob: {prob}")

In [ ]:
np.arccos(cirq.fidelity(highest_prob_state, get_ground_state_for_g(1)))

In [ ]:
cirq.fidelity(highest_prob_state, get_ground_state_for_g(1))